In [12]:
import pandas as pd
import numpy as np
import time
from datetime import datetime as dt, date, timedelta
from polygon import RESTClient
from secret import API_KEY

In [13]:
# Getting API key and client
api_key = API_KEY
client = RESTClient(api_key)

In [14]:
# Importing buy/sell signals
data = pd.read_csv('signals.csv')

In [15]:
# Creating columns for prices purchased and sold at 
data['call_prices'] = np.zeros(len(data))
data['put_prices'] = np.zeros(len(data))

In [16]:
# Setting ticker
underlying_ticker = 'AAPL'

In [17]:
def get_option_ticker(timestamp, price, type):
    # Fixing timestamps
    date = pd.to_datetime(timestamp).date()
    date_more = date + timedelta(weeks=1)

    try:
        # Initialize a list to collect all contract data
        options_list = []

        # Fetch options data
        options_data = client.list_options_contracts(
            underlying_ticker=underlying_ticker,
            expiration_date_lte=date_more,
            expiration_date_gte=date,
            strike_price_gte=(price),
            strike_price_lte=(price+1),
            contract_type=type,
            limit=1000,
            expired=True
        )

        # Loop through the options data and append each contract's data to options_list
        for option in options_data:
            options_list.append(option.__dict__)

        # Convert the list of options data to a DataFrame
        df = pd.DataFrame(options_list)

        # Finding closest ticker
        option_ticker = df['ticker'].iloc[0]

        return option_ticker
    except Exception as e:
        print(f'Error: {e}')

    return None

def get_option_data(timestamp, ticker):
    date = pd.to_datetime(timestamp).date()

    try:
        aggs = client.get_aggs(
                ticker=ticker,
                multiplier=1,
                timespan="minute",
                from_=date,
                to=date,
                limit=5000
            )
        
        df = pd.DataFrame([agg.__dict__ for agg in aggs])

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')

        df['time'] = df['timestamp'].dt.time
        df['time'] = df['time'].apply(lambda t: t.strftime("%H:%M:%S"))
        
        return df
    except Exception as e:
        print(f"Error: {e}")

    return None


In [27]:
for index, row in data.iterrows():
    if row['call_buy'] and row['call_prices'] == 0:
        timestamp = row['timestamp']
        time.sleep(12)
        ticker = get_option_ticker(timestamp, row['close'], 'call')
        time.sleep(12)
        option_price_data = get_option_data(timestamp, ticker)

        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                buy_call_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'call_prices'] = buy_call_price
                
                print(index)
                break

            except:
                print('NO DATA')
                continue

    elif row['call_sell'] and row['call_prices'] == 0:
        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                sell_call_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'call_prices'] = sell_call_price
                
                print(index)
                break

            except:
                print('NO DATA')
                continue

    elif row['put_buy'] and row['put_prices'] == 0:
        timestamp = row['timestamp']
        time.sleep(12)
        ticker = get_option_ticker(timestamp, row['close'], 'put')
        time.sleep(12)
        option_price_data = get_option_data(timestamp, ticker)

        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                buy_put_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'put_prices'] = buy_put_price
                
                print(index)
                break
                
            except:
                print('NO DATA')
                continue

    elif row['put_sell'] and row['put_prices'] == 0:
        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                sell_put_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'put_prices'] = sell_put_price
                
                print(index)
                break

            except:
                print('NO DATA')
                continue

NO DATA
NO DATA
NO DATA
Error: 'ticker'


KeyboardInterrupt: 

In [ ]:
# 19, 

,timestamp,open,high,low,close,volume,vwap,transactions,date,time,...,ema_20,atr,upper_band,lower_band,call_buy,call_sell,put_buy,put_sell,call_prices,put_prices
0,2023-01-03 11:43:00+00:00,130.7000,130.70,130.20,130.3900,4261.0,130.5428,127,2023-01-03,11:43:00,...,130.812309,0.093002,131.184318,130.440301,True,False,False,False,0.52,0.00
1,2023-01-03 11:54:00+00:00,130.1500,130.29,130.15,130.2400,3191.0,130.2187,54,2023-01-03,11:54:00,...,130.394912,0.100513,130.796965,129.992860,False,True,False,False,0.44,0.00
2,2023-01-03 14:28:00+00:00,130.8700,130.89,130.33,130.5200,53553.0,130.5298,627,2023-01-03,14:28:00,...,131.015129,0.141619,131.581604,130.448654,True,False,False,False,0.42,0.00
3,2023-01-03 14:35:00+00:00,129.8300,130.22,129.83,129.8811,502570.0,130.0497,10200,2023-01-03,14:35:00,...,130.421867,0.245869,131.405344,129.438390,False,True,False,False,0.43,0.00
4,2023-01-03 14:52:00+00:00,128.3926,128.43,127.68,127.6850,721548.0,128.0145,7831,2023-01-03,14:52:00,...,129.432444,0.363141,130.885007,127.979881,True,False,False,False,1.15,0.00
5,2023-01-03 15:23:00+00:00,125.5900,125.97,125.56,125.9400,282326.0,125.7344,3472,2023-01-03,15:23:00,...,126.217008,0.295672,127.399698,125.034318,False,True,False,False,1.10,0.00
6,2023-01-04 10:42:00+00:00,126.0200,126.02,125.91,125.9100,2477.0,125.9692,130,2023-01-04,10:42:00,...,126.261659,0.084626,126.600163,125.923155,True,False,False,False,2.18,0.00
7,2023-01-04 10:47:00+00:00,126.0500,126.10,126.04,126.1000,3531.0,126.0761,68,2023-01-04,10:47:00,...,126.182227,0.085549,126.524423,125.840031,False,True,False,False,2.46,0.00
8,2023-01-04 12:00:00+00:00,126.2500,126.25,126.03,126.1400,17896.0,126.1563,373,2023-01-04,12:00:00,...,126.381984,0.063925,126.637683,126.126285,True,False,False,False,2.34,0.00
9,2023-01-04 12:11:00+00:00,125.7700,125.95,125.77,125.9300,7441.0,125.8416,103,2023-01-04,12:11:00,...,126.048448,0.080045,126.368627,125.728269,False,True,False,False,2.56,0.00
